This notebook downloads amazon review datasets,  pre-processes the data and stores the reuqired data (the fields we will be using) in pickle format.

datasets used:  http://deepyeti.ucsd.edu/jianmo/amazon/index.html



In [1]:
import pandas as pd
import gzip
import json
import pickle

In [2]:
!wget http://deepyeti.ucsd.edu/jianmo/amazon/categoryFiles/Books.json.gz
!wget http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_Books.json.gz


--2022-06-10 00:19:22--  http://deepyeti.ucsd.edu/jianmo/amazon/categoryFiles/Books.json.gz
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11813848069 (11G) [application/octet-stream]
Saving to: ‘Books.json.gz’

Books.json.gz       100%[===================>]  11.00G  50.5MB/s    in 3m 43s  

2022-06-10 00:23:05 (50.6 MB/s) - ‘Books.json.gz’ saved [11813848069/11813848069]

--2022-06-10 00:23:05--  http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_Books.json.gz
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1219104464 (1.1G) [application/octet-stream]
Saving to: ‘meta_Books.json.gz’

meta_Books.json.gz  100%[===================>]   1.13G  48.7MB/s    in 25s     

2022-06-10

In [4]:
# process product meta data and store it in a list of python dictionary
# we can use the dictionary for loading direct into weaviate
# (i.e., no need to use pandas dataframe - covert to dictionary later)

prod_meta_data_list = [] # list of dictionaries (each dict - product meta data)

with gzip.open('meta_Books.json.gz') as f:
    for l in f:
        # load each line in the input file
        
        row = json.loads(l.strip())
        
        # pre-processing getting rid of rows with bad data
        if row['description'] ==[]:
            continue
        if (row['category'] =="") or (row['category'] ==[]):
            continue
        if row['feature'] == []:
            continue
        if (row['title'] =="") or (row['title'] ==[]):
            continue
        if not isinstance (row['asin'], str):
            continue
        # for price strip $ is any. there may still be
        # problems because people may have put comma instead of
        # period in price $ 2,40 instead of just 2.40
        try:
            rprice = float(row['price'].replace("$","").replace(",","."))
        except ValueError:
            continue
        
        # since we are storing data in picke format, we can also store
        # processed data in dictionary format (instead of pandas dataframe
        # this helps with loading into weaviate
        
        prod_meta_dict = {
            'asin' : row['asin'],
            'title' : row ['title'],
            'description' : row ['description'][0].replace("\n", " ").replace("\t", " "),
            'category' : row ['category'],
            'mainCategory' : row['main_cat'],
            'brand' : row ['brand'],
            'feature' :  row ['feature'],
            'price' : rprice
          }    
        prod_meta_data_list.append(prod_meta_dict)

print (' num products in the processed data  ', len(prod_meta_data_list))
    

 num products in the processed data   738


In [5]:
# save data in a picke file

prod_meta_fn = 'proc_prod_meta_data_june9_book.pkl'

with open(prod_meta_fn, 'wb') as fhandle:
    pickle.dump(prod_meta_data_list, fhandle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
prod_review_list = []

with gzip.open('Books.json.gz') as fp:
    for lp in fp:
        row = json.loads(lp.strip())

        if ('summary' not in row.keys()) or ('reviewText' not in row.keys()):
            continue 
        if (row['reviewText'] =="") or (row['reviewText'] ==" "):
            continue
        if (row['summary'] =="") or (row['summary'] ==" "):
            continue
        if (row['reviewerID'] =="") or (row['reviewerID'] ==" "):
            continue
        if not isinstance (row['asin'], str):
            continue
        try:
            r_overall = float(row['overall'])
        except ValueError:
            continue
            
        review_object = {
            'reviewText' : row['reviewText'].replace("\n", " ").replace("\t"," "),
            'summary' : row ['summary'],
            'overall' : r_overall,
            'reviewerid' : row ['reviewerID'],
            'reviwerName' : row.get('reviewerName', 'GENERIC'),
            'productasin' : row['asin']
          }  
        
        prod_review_list.append(review_object)
              
print (' num reviews in the processed data  ', len(prod_review_list))
 


In [6]:
import _pickle as cPickle

In [7]:
import _pickle as pickle
#import cPickle as pickle
prod_review_fn = 'proc_review_data_june9_Books.pkl'
p = pickle.Pickler(open(prod_review_fn,"wb")) 
p.fast = True 
p.dump(prod_review_list) 

In [8]:

prod_review_fn = 'proc_review_data_june9_Books.pkl'

with open(prod_review_fn, 'wb') as fh:
    pickle.dump(prod_review_list, fh)

In [9]:
print("finished")

finished
